In [6]:
from nltk.util import ngrams
def ngram(x): #傳入List
    # sentence = x
    outary = []
    for n in range(1,5):
        grams = ngrams(x, n)
        for gram in grams:
            soutstr = ''
            for i in range(0,n):
                if len(soutstr) == 0:
                    soutstr = gram[i]
                else:
                    soutstr = soutstr + ' ' + gram[i]
            outary.append(soutstr)
    return outary #回傳list

In [9]:
#coding=utf-8
import time
import pandas as pd
import numpy as np
import cPickle as pickle
import WordTransform as wt

ts = time.time()

#匯入技能原始資料
rfile=open('old\skills_all.txt','r')
raw = rfile.readlines()
rfile.close()

#變數設定
rawData = [] #原始資料 2-D list
set_skill = pickle.load(file('dic\set_skill_v8_500.data')) #技能 Set              

for each in raw[0].split(','):
    tmpList = each.split(' ')
    
    #將字解成 4-gram
    tmpList = ngram(tmpList)
    
    #相似字和錯別字轉換
    tmpList = wt.changeWord(tmpList)

    #刪除不需要的字
    tmpList = wt.deleteWord(tmpList)
    rawData.append(tmpList)
    
#為每筆資料建立dic並計算技能出現次數
                        
List_of_dic = [] # table row
zeroDic = {} #每筆資料技能Dic計算初始化
for each in set_skill:
    zeroDic[each] = 0 

for sen in rawData:
    tempDic = zeroDic.copy()
    for word in sen:
        if word in set_skill:
            tempDic[word] += 1
    List_of_dic.append(tempDic)
                        
#建立新 2-D list
list_Skill = sorted(set_skill)
newData = []
for row in List_of_dic:
    tempList = []
    for each in list_Skill:
        tempList.append(row[each])
    newData.append(tempList)

#轉為panda dataframe格式
rawDF = pd.DataFrame(newData, columns=list_Skill)



skillSum = rawDF.sum(axis=0) #這邊官方文件有錯誤 axis=0 才是計算 column sum

print skillSum

TypeError: changeWord() takes exactly 2 arguments (1 given)

In [10]:
dset = ('java script',
        'asp',
        'programming',
        'android app',
        'embedded system'
       )

In [17]:
#建立 skill的 sum字典
skillDicSum = {}
for index in range(0, len(list_Skill), 1):
    skillDicSum[list_Skill[index]]=skillSum[index]


#建立清洗 skill 的 set
skillFilter = set()    
for each in range(0, len(skillSum), 1):
    if skillSum[each] <= 500:  #只出現過一次視為極為不重要的字
        skillFilter.add(list_Skill[each])

        
#清洗
fskill_set=set_skill.difference(skillFilter)

fskill_set=fskill_set.difference(dset)


for each in skillFilter: 
    if each in skillDicSum:
        skillDicSum.pop(each)
        

for each in dset: 
    if each in skillDicSum:
        skillDicSum.pop(each)
        
tmpSkill = sorted(skillDicSum.values())

# tmpSkill.remove(' ')

# fskill_set.remove('java script')

# print len(tmpSkill)
# f = file('dic\set_skill_v8_500.data', 'w')
# pickle.dump(fskill_set,f)
# f.close()

tmpDF = pd.DataFrame.from_dict(skillDicSum.items())
pd.set_option('max_rows', None)
print len(tmpDF.index)
print tmpDF.sort(columns=1, ascending=False)


# a.to_csv('skills_tmp.csv',encoding='utf-8')

# te = time.time()
# print te-ts

26
               0     1
25          java  5395
1            sql  4897
13             c  4426
5            c++  4203
14         linux  3839
21            c#  2737
20       asp.net  2611
24           php  2408
15         mysql  2401
12    javascript  2397
6           html  2309
22       android  2100
10        jquery  1865
16           app  1522
19          .net  1409
2           ajax  1311
23           ios  1234
8   visual basic  1200
9            css   955
4            xml   854
17        c#.net   752
11           jsp   739
18        python   737
0           unix   699
7    objective-c   549
3     visual c++   532


C:\Users\BigData\Anaconda2\lib\site-packages\ipykernel\__main__.py:43: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
